In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import os

# ✅ Define Dataset Paths
BASE_DIR = r'C:\Users\lenovo\OneDrive\Desktop\E-WASTE-ANALYSIS-AND-PREDICTING-RECYCLING-METHOD\dataset\DATASET\modified-dataset'
TRAIN_PATH = os.path.join(BASE_DIR, "train")
VALID_PATH = os.path.join(BASE_DIR, "valid")
TEST_PATH = os.path.join(BASE_DIR, "test")  # For testing after training

# ✅ Image Properties
IMG_SIZE = (224, 224)  # MobileNetV2 recommended size
BATCH_SIZE = 32

# ✅ Stronger Data Augmentation to Improve Accuracy
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.6, 1.4],  
    fill_mode="nearest"
)

valid_datagen = ImageDataGenerator(rescale=1.0/255)

# ✅ Load Dataset
train_data = train_datagen.flow_from_directory(
    TRAIN_PATH, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical"
)

val_data = valid_datagen.flow_from_directory(
    VALID_PATH, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical"
)

# ✅ Load Pretrained MobileNetV2 Model
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Initially freeze the base model

# ✅ Custom Classification Head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation="relu")(x)
x = Dropout(0.4)(x)  # Increased dropout for better generalization
x = Dense(256, activation="relu")(x)
x = Dropout(0.3)(x)
output_layer = Dense(len(train_data.class_indices), activation="softmax")(x)  # Output layer

model = Model(inputs=base_model.input, outputs=output_layer)

# ✅ Compile Model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

# ✅ Callbacks
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
early_stop = EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True, verbose=1)

# ✅ Train Model
EPOCHS = 20
history = model.fit(train_data, validation_data=val_data, epochs=EPOCHS, callbacks=[reduce_lr, early_stop])

# ✅ Fine-Tune Top Layers (for Higher Accuracy)
base_model.trainable = True  # Unfreeze base model
for layer in base_model.layers[:100]:  # Keep first 100 layers frozen
    layer.trainable = False

# ✅ Recompile with Lower LR
model.compile(optimizer=Adam(learning_rate=1e-5), loss="categorical_crossentropy", metrics=["accuracy"])

# ✅ Train Again with Fine-Tuning
fine_tune_epochs = 10
history_finetune = model.fit(train_data, validation_data=val_data, epochs=fine_tune_epochs, callbacks=[reduce_lr, early_stop])

# ✅ Save Model
model.save("ewaste_mobilenetv2.h5")

print("✅ Model Training Complete & Saved as 'ewaste_mobilenetv2.h5'! 🚀")



Found 1050 images belonging to 21 classes.
Found 525 images belonging to 21 classes.


Epoch 1/20


33/33 [==============================] - 83s 2s/step - loss: 2.7013 - accuracy: 0.2152 - val_loss: 1.6624 - val_accuracy: 0.6362 - lr: 0.0010
Epoch 2/20
33/33 [==============================] - 71s 2s/step - loss: 1.7664 - accuracy: 0.4610 - val_loss: 1.1217 - val_accuracy: 0.6743 - lr: 0.0010
Epoch 3/20
33/33 [==============================] - 72s 2s/step - loss: 1.4252 - accuracy: 0.5667 - val_loss: 0.9132 - val_accuracy: 0.7429 - lr: 0.0010
Epoch 4/20
33/33 [==============================] - 76s 2s/step - loss: 1.1653 - accuracy: 0.6467 - val_loss: 0.7912 - val_accuracy: 0.7676 - lr: 0.0010
Epoch 5/20
33/33 [==============================] - 75s 2s/step - loss: 1.0663 - accuracy: 0.6676 - val_loss: 0.7566 - val_accuracy: 0.7905 - lr: 0.0010
Epoch 6/20
33/33 [==============================] - 67s 2s/step - loss: 0.9622 - accuracy: 0.7038 - val_loss: 0.6600 - val_accuracy: 0.8000 - lr:

c:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


✅ Model Training Complete & Saved as 'ewaste_mobilenetv2.h5'! 🚀
